In [1]:
from sklearn import tree
import pandas as pd
import numpy as np
import os

In [2]:
# Read the csv file into a pandas DataFrame
df = pd.read_csv('final.csv')
df.head()

,Unnamed: 0,budget,company,country,director,genre,gross,film,rating,released,...,director avg score,writer avg score,genre avg score,rating avg score,country avg score,company avg score,Golden Globes Awards,Golden Globes Nominations,Total Nominations,Total Awards
0,0,30000000,Focus Features,USA,Shane Acker,Animation,31743332,9,PG-13,1987,...,7.100000,7.100000,6.746931,6.254035,6.216790,6.839474,0.0,1.0,2.0,0.0
1,1,35000000,Columbia Pictures,USA,Robert Luketic,Crime,81159365,21,PG-13,1988,...,5.957143,6.200000,6.756322,6.254035,6.216790,6.315741,0.0,1.0,1.0,0.0
2,2,1500000,Bow and Arrow Entertainment,UK,Rob Zombie,Horror,779820,31,R,1994,...,5.666667,5.666667,5.687004,6.453213,6.660602,5.000000,0.0,1.0,1.0,0.0
3,3,40000000,Warner Bros.,USA,Brian Helgeland,Biography,95001343,42,PG-13,1995,...,6.740000,6.630000,7.040390,6.254035,6.216790,6.384354,0.0,1.0,1.0,0.0
4,4,13000000,Dollface,USA,Mark Christopher,Drama,16574731,54,R,1999,...,5.800000,5.800000,6.715720,6.453213,6.216790,5.800000,0.0,1.0,1.0,0.0


In [55]:
clean = df[['movie score', 'budget', 'gross','runtime', 'votes','star avg score', 'director avg score','writer avg score','genre avg score','rating avg score','company avg score','country avg score','Total Nominations', 'Total Awards']]
clean.head(3)

,movie score,budget,gross,runtime,votes,star avg score,director avg score,writer avg score,genre avg score,rating avg score,company avg score,country avg score,Total Nominations,Total Awards
0,7.1,30000000,31743332,79,117268,6.864286,7.100000,7.100000,6.746931,6.254035,6.839474,6.216790,2.0,0.0
1,6.8,35000000,81159365,123,201705,6.875000,5.957143,6.200000,6.756322,6.254035,6.315741,6.216790,1.0,0.0
2,5.0,1500000,779820,102,11945,5.900000,5.666667,5.666667,5.687004,6.453213,5.000000,6.660602,1.0,0.0


In [56]:
features = pd.get_dummies(clean)

In [57]:
labels = np.array(clean['movie score'])
clean= clean.drop('movie score', axis = 1)
clean_list = list(clean.columns)
clean = np.array(clean)

In [58]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_clean, test_clean, train_labels, test_labels = train_test_split(clean, labels, test_size = 0.2, random_state = 42)

In [59]:
print('Training Features Shape:', train_clean.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_clean.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (5456, 13)
Training Labels Shape: (5456,)
Testing Features Shape: (1364, 13)
Testing Labels Shape: (1364,)


In [60]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 200 decision trees
rf = RandomForestRegressor(n_estimators = 200, random_state = 42)
# Train the model on training data
rf.fit(train_clean, train_labels);

In [61]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_clean)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'points.')


Mean Absolute Error: 0.24 points.


In [62]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 96.04 %.


In [63]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]
# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = clean_list, rounded = True, precision = 1)
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')
# Write graph to a png file
graph.write_png('tree.png')

In [64]:
# Limit depth of tree to 3 levels
rf_small = RandomForestRegressor(n_estimators=10, max_depth = 3)
rf_small.fit(train_clean, train_labels)
# Extract the small tree
tree_small = rf_small.estimators_[5]
# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = clean_list, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
graph.write_png('small_tree.png');

In [65]:
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(clean_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: writer avg score     Importance: 0.75
Variable: star avg score       Importance: 0.06
Variable: director avg score   Importance: 0.06
Variable: votes                Importance: 0.04
Variable: company avg score    Importance: 0.03
Variable: budget               Importance: 0.02
Variable: gross                Importance: 0.01
Variable: runtime              Importance: 0.01
Variable: genre avg score      Importance: 0.01
Variable: Total Nominations    Importance: 0.01
Variable: rating avg score     Importance: 0.0
Variable: country avg score    Importance: 0.0
Variable: Total Awards         Importance: 0.0
